# Imports

In [ ]:
import os
import sys
from dotenv import load_dotenv
from matplotlib import pyplot as plt
import pandas as pd
from transformers import pipeline
import json
import requests

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    !pip install gigachat

from gigachat import GigaChat

load_dotenv()
SBER_API = os.environ.get("SBER", "")
PROXY_API = os.environ.get("PROXYAPI", "")

# Для более качественных графиков
%config InlineBackend.figure_format='retina'
plt.rcParams["figure.dpi"] = 150

# Huggingface

[Документация](https://huggingface.co/docs/transformers/quicktour)

In [ ]:
classifier = pipeline("sentiment-analysis")

In [ ]:
classifier("We are very happy to show you the 🤗 Transformers library.")

Можем довольно просто получить эмбеддинг текста

In [ ]:
embed_model = pipeline("feature-extraction")

embed_model("Oh hi mark")

Задание: 
- выделить трейн и тест выборки на датасете [anime_reviews.csv](https://drive.google.com/file/d/1osuUDbfK8e8IO5VIInwgg8dOzyYNEuF2/view?usp=sharing), доля теста 0.2, random_state 42
- сделать предсказание тональности на тестовой выборке при помощи [модели](https://huggingface.co/tabularisai/multilingual-sentiment-analysis), посчитать f1

In [ ]:
df = pd.read_csv("data/anime_reviews.csv")
df

Аналогично эмбеддингам word2vec, эмбеддинги берта можно использовать как признаки для обучения

# Использование LLM для разметки

## Бесплатный способ

Нужно получить authentication key на [сайте](https://developers.sber.ru/studio)

In [ ]:
def ask_question_free(prompt):
    giga = GigaChat(credentials=SBER_API, model="GigaChat", verify_ssl_certs=False)
    return giga.chat(prompt).choices[0].message.content


ask_question_free("is amogus a good game?")

## Платный способ

[Ссылка на сервис](https://proxyapi.ru/)

In [ ]:
def ask_question(prompt, key):
    url = "https://api.proxyapi.ru/openai/v1/chat/completions"
    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {key}"}
    data = {"model": "gpt-4o-mini", "messages": [{"role": "user", "content": prompt}]}
    response = requests.post(url, headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        return [d["message"]["content"] for d in response.json()["choices"]]

    raise requests.RequestException(response.status_code, response.text)


ask_question("Can you do Tommy Wiseau impression?", PROXY_API)

## Разметка

Задача: ниже взята часть датасета с обзорами, используя запросы к llm, сделайте предсказание (негативный класс или позитивный) на этой выборке, посчитайте f1.
Попробуйте улучшить качество при помощи улучшения промпта

In [ ]:
df = pd.read_csv("data/anime_reviews.csv")
df_sample = df.sample(10, random_state=2)